In [15]:
# 단순선형회귀
# 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 변수 선언 
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
print(x_train, x_train.shape)
print(y_train, y_train.shape)

# 가중치와 편향 0으로 초기화 -> 현재의 방정식 y=0*x+0
w = torch.zeros(1, requires_grad =True) 
print(w)
b = torch.zeros(1, requires_grad=True)
print(b)
# requires_grad는 이 변수가 학습하면서 변경되는 것임을 의미 -> backward 함수를 호출하면 자동으로 미분 계산됨

# 비용 함수(평균 제곱 오차) 선언
y_pred = x_train*w + b
print(y_pred)
cost = torch.mean((y_pred-y_train)**2)
print(cost)

# 경사하강법 구현 (SGD 사용)
optimizer = optim.SGD([w,b], lr=0.01) 
num_epochs = 1000
for epoch in range(num_epochs+1):
    
    # 가설 설정
    y_pred = x_train*w + b  
    
    # 비용 함수(평균 제곱 오차) 계산
    cost = torch.mean((y_pred-y_train)**2)  
    
    # cost로 y_pred 개선
    optimizer.zero_grad()  # 파이토치는 이전의 gradient값에 누적하기 때문에 gradient 0으로 초기화해야 함 
    cost.backward()  # 비용함수 미분하여 w와 b에 대한 gradient 계산
    optimizer.step()  # w와 b 업데이트

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f} w_gradient: {:.5f} b_gradient: {:.5f}'.format(
            epoch, num_epochs, w.item(), b.item(), cost.item(), w.grad.item(), b.grad.item()
        ))

tensor([[1.],
        [2.],
        [3.]]) torch.Size([3, 1])
tensor([[2.],
        [4.],
        [6.]]) torch.Size([3, 1])
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)
tensor(18.6667, grad_fn=<MeanBackward0>)
Epoch    0/1000 W: 0.187, b: 0.080 Cost: 18.666666 w_gradient: -18.66667 b_gradient: -8.00000
Epoch  100/1000 W: 1.746, b: 0.578 Cost: 0.048171 w_gradient: -0.06142 b_gradient: 0.13924
Epoch  200/1000 W: 1.800, b: 0.454 Cost: 0.029767 w_gradient: -0.04817 b_gradient: 0.10950
Epoch  300/1000 W: 1.843, b: 0.357 Cost: 0.018394 w_gradient: -0.03787 b_gradient: 0.08608
Epoch  400/1000 W: 1.876, b: 0.281 Cost: 0.011366 w_gradient: -0.02977 b_gradient: 0.06767
Epoch  500/1000 W: 1.903, b: 0.221 Cost: 0.007024 w_gradient: -0.02340 b_gradient: 0.05319
Epoch  600/1000 W: 1.924, b: 0.174 Cost: 0.004340 w_gradient: -0.01839 b_gradient: 0.04181
Epoch  700/1000 W: 1.940, b: 0.136 Cost: 0.002682 w_gradient: 

In [29]:
# 단순 선형 회귀 - nn.Module로 구현

'''
위에서는 가설, 비용함수를 직접 정의하여 모델을 구현함
이번에는 이미 있는 함수를 불러와 모델을 구현할 것임

- nn.Linear(input_dim, output_dim): 선형 회귀 모델
- nn.functional.mse_loss(prediction, y_train): 평균 제곱 오차
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# 모델 선언 및 초기화
model = nn.Linear(1,1)  # 단순 선형 회귀이므로 input_dim=1, output_dim=1
print(list(model.parameters()))  # .parameters()함수로 가중치w와 편향 b를 불러오기
# 두값은 모두 랜덤하게 초기화 됨, 두값 모두 학습의 대상이므로 requires_grad=Ture로 되어 있음

# optimizer 정의
optimizer = optim.SGD(model.parameters(), lr=0.01)  # .parameters()로 w와 b 전달

num_epochs = 1000
for epoch in range(num_epochs+1):
    
    # 가설
    y_pred = model(x_train)
    
    # 비용 계산
    cost = F.mse_loss(y_pred, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()  # gradient 0으로 초기화
    cost.backward()  # gradient 계산 
    optimizer.step()  # w와 b 업데이트
    
    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, num_epochs, cost.item()
      ))
    
# x에 임의의 값을 넣어 모델이 예측하는 y값 확인하기
new_value = torch.FloatTensor([[4.0]])  # 임의의 값 4를 선언
y_hat = model(new_value)  # forward 연산
print('예측값: ', y_hat)

# 학습 후의 w, b 확인
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]
Epoch    0/1000 Cost: 13.103541
Epoch  100/1000 Cost: 0.002791
Epoch  200/1000 Cost: 0.001724
Epoch  300/1000 Cost: 0.001066
Epoch  400/1000 Cost: 0.000658
Epoch  500/1000 Cost: 0.000407
Epoch  600/1000 Cost: 0.000251
Epoch  700/1000 Cost: 0.000155
Epoch  800/1000 Cost: 0.000096
Epoch  900/1000 Cost: 0.000059
Epoch 1000/1000 Cost: 0.000037
예측값:  tensor([[7.9879]], grad_fn=<AddmmBackward>)
[Parameter containing:
tensor([[1.9930]], requires_grad=True), Parameter containing:
tensor([0.0159], requires_grad=True)]


In [35]:
# 단순 선형 회귀 - 모델을 클래스로 구현하기

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# torch.nn.Module을 상속받는 파이썬 클래스
class LinearRegressionModel(nn.Module):
    def __init__(self):  # 생성자 정의 (객체가 생성될때 자동으로 호출되는 메서드)
        super().__init__()  # nn.Module 클래스의 속성들을 가지고 초기화
        self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim=1, output_dim=1
    
    # model(입력데이터) 하면 자동으로 실행되는 forward 함수 정의
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

model = LinearRegressionModel()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

num_epochs = 2000
for epoch in range(num_epochs):
    y_pred = model(x_train)
    
    cost = F.mse_loss(y_pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, num_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 18.562185
Epoch  100/2000 Cost: 0.128051
Epoch  200/2000 Cost: 0.079128
Epoch  300/2000 Cost: 0.048896
Epoch  400/2000 Cost: 0.030215
Epoch  500/2000 Cost: 0.018671
Epoch  600/2000 Cost: 0.011538
Epoch  700/2000 Cost: 0.007129
Epoch  800/2000 Cost: 0.004406
Epoch  900/2000 Cost: 0.002722
Epoch 1000/2000 Cost: 0.001682
Epoch 1100/2000 Cost: 0.001040
Epoch 1200/2000 Cost: 0.000642
Epoch 1300/2000 Cost: 0.000397
Epoch 1400/2000 Cost: 0.000245
Epoch 1500/2000 Cost: 0.000152
Epoch 1600/2000 Cost: 0.000094
Epoch 1700/2000 Cost: 0.000058
Epoch 1800/2000 Cost: 0.000036
Epoch 1900/2000 Cost: 0.000022


In [19]:
# 다중 선형 회귀

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 가중치 초기화
w1 = torch.zeros(1, requires_grad =True)
w2 = torch.zeros(1, requires_grad =True)
w3 = torch.zeros(1, requires_grad =True)
b = torch.zeros(1, requires_grad =True)

# optimizer 설정 
optimizer = optim.SGD([w1,w2,w2,b], lr=1e-5)

num_epochs = 1000
for epoch in range(num_epochs+1):
    # 가설 세우기
    y_pred = w1*x1_train + w2*x2_train + w3*x3_train + b
    
    # 비용함수 계산
    cost = torch.mean((y_pred-y_train)**2)
    
    # gradient 계산 및 업데이트
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, num_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.587 w3: 0.000 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.696 w2: 1.323 w3: 0.000 b: 0.008 Cost: 11.417717
Epoch  200/1000 w1: 0.718 w2: 1.300 w3: 0.000 b: 0.008 Cost: 10.682022
Epoch  300/1000 w1: 0.740 w2: 1.279 w3: 0.000 b: 0.008 Cost: 9.995893
Epoch  400/1000 w1: 0.760 w2: 1.258 w3: 0.000 b: 0.008 Cost: 9.356130
Epoch  500/1000 w1: 0.780 w2: 1.238 w3: 0.000 b: 0.008 Cost: 8.759562
Epoch  600/1000 w1: 0.799 w2: 1.219 w3: 0.000 b: 0.009 Cost: 8.203227
Epoch  700/1000 w1: 0.818 w2: 1.201 w3: 0.000 b: 0.009 Cost: 7.684474
Epoch  800/1000 w1: 0.836 w2: 1.183 w3: 0.000 b: 0.009 Cost: 7.200711
Epoch  900/1000 w1: 0.853 w2: 1.165 w3: 0.000 b: 0.009 Cost: 6.749591
Epoch 1000/1000 w1: 0.870 w2: 1.149 w3: 0.000 b: 0.009 Cost: 6.328963


In [25]:
# 다중 선형 회귀 - 벡터와 행렬연산

# 훈련 데이터 행렬로 선언하기 - 한번에 모든 샘플을 선언
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  80], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 가중치와 편향 초기화
W = torch.zeros((3,1), requires_grad=True)  # W를 행렬로 선언 
print(W.shape)
b = torch.zeros(1, requires_grad=True)
print(b.shape)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=1e-5)

num_epochs = 20
for epoch in range(num_epochs):
    # 행렬 곱셈으로 가설 선언 
    y_pred = x_train.matmul(W) + b  # b는 브로트캐스팅되어 더해짐
    
    # cost 함수 정의
    cost = torch.mean((y_pred-y_train)**2)
    
    # gradient 계산 및 업데이트
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, num_epochs, y_pred.squeeze().detach(), cost.item()
    ))

torch.Size([3, 1])
torch.Size([1])
Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]) Cost: 9537.694336
Epoch    2/20 hypothesis: tensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]) Cost: 3069.590088
Epoch    3/20 hypothesis: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]) Cost: 990.670898
Epoch    4/20 hypothesis: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]) Cost: 322.482056
Epoch    5/20 hypothesis: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]) Cost: 107.717064
Epoch    6/20 hypothesis: tensor([148.9423, 178.9730, 169.8976, 192.0301, 136.5279]) Cost: 38.687500
Epoch    7/20 hypothesis: tensor([151.1574, 181.6346, 172.4254, 194.8856, 138.5585]) Cost: 16.499043
Epoch    8/20 hypothesis: tensor([152.4131, 183.1435, 173.8590, 196.5043, 139.7097]) Cost: 9.365657
Epoch    9/20 hypothesis: tensor([153.1250, 183.9988, 174.6723, 197.4217, 140.3625

In [34]:
# 다중 선형 회귀 - nn.Module로 구현

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 선언 및 초기화
model = nn.Linear(3,1)  # 다중 선형 회귀이므로 input_dim=3, output_dim=1
print(list(model.parameters()))

# optimizer 선언
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

num_epochs = 2000
for epoch in range(num_epochs):
    
    # 가설 설정
    y_pred = model(x_train)
    
    # 비용 계산
    cost = F.mse_loss(y_pred, y_train)
    
    # gradient 계산 
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, num_epochs, cost.item()
      ))
        
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
y_hat = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", y_hat) 

# 학습 후의 3개의 w와 b의 값을 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]
Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210559
Epoch 1100/2000 Cost: 0.209230
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206762
Epoch 1400/2000 Cost: 0.205618
Epoch 1500/2000 Cost: 0.204529
Epoch 1600/2000 Cost: 0.203481
Epoch 1700/2000 Cost: 0.202486
Epoch 1800/2000 Cost: 0.201539
Epoch 1900/2000 Cost: 0.200634
훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.2305]], grad_fn=<AddmmBackward>)
[Parameter containing:
tensor([[0.9778, 0.4539, 0.5768]], requires_grad=True), Parameter containing:
tensor([0.2802], requires_grad=True)]


In [36]:
# 다중 선형 회귀 - 모델을 클래스로 구현하기

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

class MultiLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1
        
    def forward(self, x):
        return self.linear(x)
    
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
    
model = MultiLinearRegression()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

num_epochs = 2000
for epoch in range(num_epochs):
    y_pred = model(x_train)
    cost = F.mse_loss(y_pred, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, num_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210559
Epoch 1100/2000 Cost: 0.209230
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206762
Epoch 1400/2000 Cost: 0.205618
Epoch 1500/2000 Cost: 0.204529
Epoch 1600/2000 Cost: 0.203481
Epoch 1700/2000 Cost: 0.202486
Epoch 1800/2000 Cost: 0.201539
Epoch 1900/2000 Cost: 0.200634


In [48]:
# 다중 선형 회귀 - 미니 배치 경사 하강법 사용

'''
미니 배치: 전체 데이터를 작은 단위로 나우어 해당 단위로 학습을 진행
미니 배치만큼 사용해서 그에 대한 비용을 계산 경사하강법 수행

1 epoch: 전체 데이터를 한 번 학습
1 iteration: 미니 배치를 한 번 학습

DataLoader를 사용하면 미니 배치 학습, 데이터 셔플, 병렬 처리를 간단하게 수행 가능 
기본적으로 Dataset을 정의하고 dataLoader에 전달하면 됨 
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# DataLoader를 사용하기 위해 데이터 만들기
dataset = TensorDataset(x_train, y_train)
print(list(dataset))

# shuffle=True: Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿈
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
print(list(dataloader))

# 모델과 optimizer 초기화
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

# 훈련 진행
num_epochs = 20
for epoch in range(num_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx+1, samples)
        x_train, y_train = samples
        y_pred = model(x_train)
        
        cost = F.mse_loss(y_pred, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, num_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

[(tensor([73., 80., 75.]), tensor([152.])), (tensor([93., 88., 93.]), tensor([185.])), (tensor([89., 91., 90.]), tensor([180.])), (tensor([ 96.,  98., 100.]), tensor([196.])), (tensor([73., 66., 70.]), tensor([142.]))]
[[tensor([[73., 66., 70.],
        [73., 80., 75.]]), tensor([[142.],
        [152.]])], [tensor([[89., 91., 90.],
        [93., 88., 93.]]), tensor([[180.],
        [185.]])], [tensor([[ 96.,  98., 100.]]), tensor([[196.]])]]
Epoch    0/20 Batch 1/3 Cost: 8742.169922
Epoch    0/20 Batch 2/3 Cost: 1625.443604
Epoch    0/20 Batch 3/3 Cost: 401.292236
Epoch    1/20 Batch 1/3 Cost: 251.589279
Epoch    1/20 Batch 2/3 Cost: 88.792755
Epoch    1/20 Batch 3/3 Cost: 22.403616
Epoch    2/20 Batch 1/3 Cost: 6.824500
Epoch    2/20 Batch 2/3 Cost: 3.784405
Epoch    2/20 Batch 3/3 Cost: 3.514938
Epoch    3/20 Batch 1/3 Cost: 0.979391
Epoch    3/20 Batch 2/3 Cost: 0.751226
Epoch    3/20 Batch 3/3 Cost: 0.281757
Epoch    4/20 Batch 1/3 Cost: 1.071533
Epoch    4/20 Batch 2/3 Cost: 0.726

In [52]:
# 다중 선형 회귀 - custom dataset으로 구현

'''
torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만드는 경우

class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):
  데이터셋의 전처리를 해주는 부분

  def __len__(self):
  데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

  def __getitem__(self, idx): 
  데이터셋에서 특정 1개의 샘플을 가져오는 함수
  
len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item
'''

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Dataset 상속
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
        
    # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.x_data)
    
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 tensor 형태로 리턴
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

num_epochs = 20
for epoch in range(num_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx)
        # print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))

Epoch    0/20 Batch 1/3 Cost: 57738.265625
Epoch    0/20 Batch 2/3 Cost: 7475.723633
Epoch    0/20 Batch 3/3 Cost: 5112.232422
Epoch    1/20 Batch 1/3 Cost: 1371.436035
Epoch    1/20 Batch 2/3 Cost: 349.975830
Epoch    1/20 Batch 3/3 Cost: 59.614677
Epoch    2/20 Batch 1/3 Cost: 43.241051
Epoch    2/20 Batch 2/3 Cost: 18.453762
Epoch    2/20 Batch 3/3 Cost: 4.698302
Epoch    3/20 Batch 1/3 Cost: 3.978527
Epoch    3/20 Batch 2/3 Cost: 0.036147
Epoch    3/20 Batch 3/3 Cost: 0.102473
Epoch    4/20 Batch 1/3 Cost: 1.630585
Epoch    4/20 Batch 2/3 Cost: 0.299462
Epoch    4/20 Batch 3/3 Cost: 0.014753
Epoch    5/20 Batch 1/3 Cost: 1.361493
Epoch    5/20 Batch 2/3 Cost: 0.461822
Epoch    5/20 Batch 3/3 Cost: 0.012013
Epoch    6/20 Batch 1/3 Cost: 0.098629
Epoch    6/20 Batch 2/3 Cost: 0.027262
Epoch    6/20 Batch 3/3 Cost: 2.994628
Epoch    7/20 Batch 1/3 Cost: 0.481490
Epoch    7/20 Batch 2/3 Cost: 1.011021
Epoch    7/20 Batch 3/3 Cost: 1.130528
Epoch    8/20 Batch 1/3 Cost: 0.103577
Epoch  